In [1]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [23]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [24]:
X_train[0].shape

(32, 32, 3)

In [25]:
keras.backend.clear_session()

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 activation="elu",
                                 kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [27]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)

callbacks = [early_stopping_cb, model_checkpoint_cb]

In [26]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)

model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = optimizer,
              metrics = ["accuracy"])

In [28]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 25s 15ms/step - loss: 4.5394 - accuracy: 0.1656 - val_loss: 2.1997 - val_accuracy: 0.2198
Epoch 2/100
1407/1407 [==============================] - 21s 15ms/step - loss: 2.0819 - accuracy: 0.2430 - val_loss: 2.0238 - val_accuracy: 0.2654
Epoch 3/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.9510 - accuracy: 0.2880 - val_loss: 1.9242 - val_accuracy: 0.2932
Epoch 4/100
1407/1407 [==============================] - 22s 16ms/step - loss: 1.8637 - accuracy: 0.3240 - val_loss: 1.8202 - val_accuracy: 0.3352
Epoch 5/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.7945 - accuracy: 0.3503 - val_loss: 1.7645 - val_accuracy: 0.3620
Epoch 6/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.7378 - accuracy: 0.3736 - val_loss: 1.7244 - val_accuracy: 0.3680
Epoch 7/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.6938 - accuracy: 0.3901 - val_loss: 1

## 이전의 모델에서 배치정규화를 추가시켜 정확도 비교

In [29]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 activation="elu",
                                 kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10, activation="softmax"))

In [30]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Nadam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
              metrics=["accuracy"])

In [32]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
callbacks = [early_stopping_cb, model_checkpoint_cb]

In [33]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 52s 37ms/step - loss: 1.6834 - accuracy: 0.4020 - val_loss: 1.6104 - val_accuracy: 0.4258
Epoch 2/100
1407/1407 [==============================] - 52s 37ms/step - loss: 1.6170 - accuracy: 0.4279 - val_loss: 1.5546 - val_accuracy: 0.4506
Epoch 3/100
1407/1407 [==============================] - 53s 38ms/step - loss: 1.5683 - accuracy: 0.4436 - val_loss: 1.4899 - val_accuracy: 0.4700
Epoch 4/100
1407/1407 [==============================] - 53s 38ms/step - loss: 1.5206 - accuracy: 0.4594 - val_loss: 1.4425 - val_accuracy: 0.4872
Epoch 5/100
1407/1407 [==============================] - 53s 38ms/step - loss: 1.4845 - accuracy: 0.4750 - val_loss: 1.4462 - val_accuracy: 0.4884
Epoch 6/100
1407/1407 [==============================] - 52s 37ms/step - loss: 1.4554 - accuracy: 0.4842 - val_loss: 1.4077 - val_accuracy: 0.5034
Epoch 7/100
1407/1407 [==============================] - 53s 38ms/step - loss: 1.4224 - accuracy: 0.4965 - val_loss: 1

#### 배치정규화를 추가하여 학습을 진행하니 최종 val_accuracy가 0.4790에서 0.5524로 증가하였다.

#### 단, 배치정규화를 사용하니 학습에 필요한 시간이 증가하였다.

## 배치정규화를 사용하지 않고 selu로 다시 학습

In [38]:
keras.backend.clear_session()

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [39]:
optimizer = keras.optimizers.Nadam(learning_rate=7e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [40]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5", save_best_only=True)

callbacks = [early_stopping_cb, model_checkpoint_cb]

In [41]:
# SELU를 사용하기 위해서는 네트워크가 자기 정규화가 되어야 함. 때문에 데이터셋을 정규화함.

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

In [42]:
model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 25s 15ms/step - loss: 1.9165 - accuracy: 0.3157 - val_loss: 1.8406 - val_accuracy: 0.3210
Epoch 2/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.7054 - accuracy: 0.3970 - val_loss: 1.7119 - val_accuracy: 0.3832
Epoch 3/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.6112 - accuracy: 0.4322 - val_loss: 1.6513 - val_accuracy: 0.4172
Epoch 4/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.5407 - accuracy: 0.4563 - val_loss: 1.6156 - val_accuracy: 0.4292
Epoch 5/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.4805 - accuracy: 0.4809 - val_loss: 1.5978 - val_accuracy: 0.4468
Epoch 6/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.4297 - accuracy: 0.4967 - val_loss: 1.5211 - val_accuracy: 0.4644
Epoch 7/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.3924 - accuracy: 0.5146 - val_loss: 1

#### 배치정규화를 사용했을 경우보다 학습 시간은 줄었지만, 정확도는 낮아졌음을 확인할 수 있다.

## 드롭아웃을 사용하여 모델을 학습했을 때와의 비교

##### 모든 층에서 드롭아웃을 사용했을 때 과하게 언더피팅되는 것을 확인함. 때문에 마지막 층 이전에서만 드롭아웃층을 사용함.

In [59]:
keras.backend.clear_session()

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.AlphaDropout(rate=0.2))
for _ in range(20):
  model.add(keras.layers.Dense(100,
                               kernel_initializer="lecun_normal",
                               activation="selu"))
model.add(keras.layers.AlphaDropout(rate=0.2))
model.add(keras.layers.Dense(10, activation="softmax"))

In [60]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-4)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [61]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_alpha_dropout_model.h5", save_best_only=True)

callbacks = [early_stopping_cb, model_checkpoint_cb]

In [62]:
X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

In [63]:
model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 26s 15ms/step - loss: 1.9323 - accuracy: 0.3116 - val_loss: 1.7940 - val_accuracy: 0.3474
Epoch 2/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.7384 - accuracy: 0.3841 - val_loss: 1.8249 - val_accuracy: 0.3964
Epoch 3/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.6633 - accuracy: 0.4146 - val_loss: 1.8089 - val_accuracy: 0.3962
Epoch 4/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.6102 - accuracy: 0.4331 - val_loss: 1.6201 - val_accuracy: 0.4554
Epoch 5/100
1407/1407 [==============================] - 22s 15ms/step - loss: 1.5674 - accuracy: 0.4494 - val_loss: 1.7197 - val_accuracy: 0.4296
Epoch 6/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.5333 - accuracy: 0.4623 - val_loss: 1.7114 - val_accuracy: 0.4420
Epoch 7/100
1407/1407 [==============================] - 22s 15ms/step - loss: 1.5001 - accuracy: 0.4750 - val_loss: 1

#### 드롭아웃을 사용하지 않았을 경우보다 정확도가 증가함

## 학습한 모델을 MCDropout으로 바꾸어 예측했을 때의 차이 비교

In [64]:
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [65]:
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

In [66]:
def mc_dropout_predict_probas(mc_model, X, n_samples=10):
    Y_probas = [mc_model.predict(X) for sample in range(n_samples)]
    return np.mean(Y_probas, axis=0)

def mc_dropout_predict_classes(mc_model, X, n_samples=10):
    Y_probas = mc_dropout_predict_probas(mc_model, X, n_samples)
    return np.argmax(Y_probas, axis=1)

In [67]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

y_pred = mc_dropout_predict_classes(mc_model, X_valid_scaled)
accuracy = np.mean(y_pred == y_valid[:, 0])
accuracy

0.5146

#### 단순 드롭아웃을 사용했을 때보다 validation dataset에서의 정확도가 증가했다.

In [68]:
keras.backend.clear_session()

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.AlphaDropout(rate=0.2))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
  model.add(keras.layers.Dense(100,
                               kernel_initializer="lecun_normal",
                               activation="selu"))
  model.add(keras.layers.BatchNormalization())
model.add(keras.layers.AlphaDropout(rate=0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10, activation="softmax"))

In [69]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-4)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [70]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_alpha_dropout_model.h5", save_best_only=True)

callbacks = [early_stopping_cb, model_checkpoint_cb]

In [ ]:
model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)